**Resumen**

Este estudio analiza un conjunto de datos detallados sobre vuelos, que incluye información sobre fechas de salida, clasificación de vuelos (cabotaje o internacional), aerolíneas, aeropuertos, localidades de origen y destino, y la cantidad de pasajeros y asientos disponibles por vuelo. El análisis permite identificar patrones de tráfico aéreo a nivel anual, la cantidad de pasajeros, la capacidad de los vuelos y la conectividad entre distintos puntos de Argentina y el resto del mundo. Se busca evaluar la frecuencia de vuelos por aeropuerto, la popularidad de diferentes destinos, y la clasificación de vuelos más utilizada.

**Objetivo**

El objetivo de este estudio es determinar cuál es la aerolínea con mayor tráfico y mayor cantidad de vuelos en el conjunto de datos, identificar cuál es el vuelo con más pasajeros y establecer cuál es el país con más tránsito aéreo. Estos hallazgos proporcionarán información clave para optimizar las operaciones, mejorar la oferta de vuelos y tomar decisiones estratégicas sobre expansión y concentración de recursos en rutas de alto volumen.

**Contexto Comercial**

En el contexto de una aerolínea argentina que busca optimizar su red de vuelos y aumentar la eficiencia operativa, entender los patrones de tráfico aéreo y la distribución de pasajeros por rutas nacionales e internacionales es esencial. La empresa, con un enfoque en la mejora de la conectividad y la expansión de su red, busca maximizar la utilización de sus aeropuertos y recursos. Este análisis de datos contribuirá a la toma de decisiones estratégicas, como la asignación de vuelos a destinos más demandados, la evaluación de la capacidad de los aeropuertos, y la identificación de oportunidades para ampliar la oferta en mercados internacionales clave.

*Hipotesis*

Existe una correlación significativa entre las variables X (clasificación de vuelo, clase de vuelo, aerolínea, origen y destino por país y continente, número de asientos y vuelos) y Y (número de pasajeros), lo que sugiere una relación causal entre los factores analizados. Esto indica que las características de los vuelos (como la clase, aerolínea, la cantidad de asientos, y los destinos) pueden influir de manera significativa en el número de pasajeros transportados.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = pd.read_excel("/content/drive/MyDrive/Colab/dbvuelos.xltx")
df.shape

(832883, 21)

In [6]:
df.info()
df.describe()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 832883 entries, 0 to 832882
Data columns (total 21 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   indice_tiempo        832883 non-null  datetime64[ns]
 1   clasificacion_vuelo  832883 non-null  object        
 2   clase_vuelo          832883 non-null  object        
 3   aerolinea            832883 non-null  object        
 4   origen_oaci          832883 non-null  object        
 5   origen_aeropuerto    832883 non-null  object        
 6   origen_localidad     832883 non-null  object        
 7   origen_provincia     658763 non-null  object        
 8   origen_pais          832883 non-null  object        
 9   origen_continente    832882 non-null  object        
 10  destino_oaci         832883 non-null  object        
 11  destino_aeropuerto   832883 non-null  object        
 12  destino_localidad    832883 non-null  object        
 13  destino_provin

,indice_tiempo,clasificacion_vuelo,clase_vuelo,aerolinea,origen_oaci,origen_aeropuerto,origen_localidad,origen_provincia,origen_pais,origen_continente,...,destino_aeropuerto,destino_localidad,destino_provincia,destino_pais,destino_continente,pasajeros,asientos,vuelos,Unnamed: 19,Unnamed: 20
0,2017-01-01,Cabotaje,No Regular,Andes Líneas Aéreas,SAAV,Aeropuerto de Sauce Viejo,Santa Fe,Santa Fe,Argentina,América del Sur,...,Aeropuerto Int. Tte. Luis Candelaria,San Carlos de Bariloche,Río Negro,Argentina,América del Sur,85,85,1,NaN,NaN
1,2017-01-01,Cabotaje,No Regular,Andes Líneas Aéreas,SASA,Aeropuerto Int. Martín Miguel de Güemes,Salta,Salta,Argentina,América del Sur,...,Aeropuerto Int. Gdor. Horacio Guzmán,San Salvador de Jujuy,Jujuy,Argentina,América del Sur,23,83,1,NaN,NaN
2,2017-01-01,Cabotaje,No Regular,Andes Líneas Aéreas,SAZS,Aeropuerto Int. Tte. Luis Candelaria,San Carlos de Bariloche,Río Negro,Argentina,América del Sur,...,Aeropuerto de Sauce Viejo,Santa Fe,Santa Fe,Argentina,América del Sur,78,83,1,NaN,NaN
3,2017-01-01,Cabotaje,Regular,Aerolíneas Argentinas,SAAR,Aeropuerto Int. de Rosario Islas Malvinas,Rosario,Santa Fe,Argentina,América del Sur,...,Aeropuerto Int. Ingeniero Ambrosio Taravella,Córdoba,Córdoba,Argentina,América del Sur,48,128,1,NaN,NaN
4,2017-01-01,Cabotaje,Regular,Aerolíneas Argentinas,SAAR,Aeropuerto Int. de Rosario Islas Malvinas,Rosario,Santa Fe,Argentina,América del Sur,...,Aeropuerto Int. Ministro Pistarini,Ezeiza,Buenos Aires,Argentina,América del Sur,90,128,1,NaN,NaN


Descripción de las Variables

**indice_tiempo:** Fechas o periodos en los que se registraron los vuelos.

**clasificacion_vuelo:** Categoría del vuelo (por ejemplo, cabotaje o internacional).

**clase_vuelo:** Tipo de clase de los pasajeros (e.g., económica, ejecutiva).

**aerolinea:** La aerolínea que opera el vuelo.

**origen_oaci, destino_oaci:** Códigos de la OACI para los aeropuertos de origen y destino.

**origen_localidad, destino_localidad:** Localidades de origen y destino.

**pasajeros:** Número de pasajeros en cada vuelo.

**asientos:** Número de asientos disponibles.

**vuelos:** Número de vuelos operados.

In [ ]:
df.groupby('aerolinea')['pasajeros'].sum()

In [7]:
print(df.isnull().sum())

indice_tiempo               0
clasificacion_vuelo         0
clase_vuelo                 0
aerolinea                   0
origen_oaci                 0
origen_aeropuerto           0
origen_localidad            0
origen_provincia       174120
origen_pais                 0
origen_continente           1
destino_oaci                0
destino_aeropuerto          0
destino_localidad           0
destino_provincia      175091
destino_pais              198
destino_continente          0
pasajeros                   0
asientos                    0
vuelos                      0
Unnamed: 19            795163
Unnamed: 20            832165
dtype: int64


In [ ]:
df.fillna(df.mean(), inplace=True)

In [ ]:
X = df[['clasificacion_vuelo', 'clase_vuelo', 'aerolinea', 'origen_pais', 'origen_continente', 'destino_pais', 'destino_continente', 'asientos', 'vuelos']]

y = df['pasajeros']

NameError: name 'df' is not defined

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
modelo = KNeighborsClassifier(n_neighbors=10)
modelo.fit(X_train, y_train)

In [ ]:
preds_train = modelo.predict(X_train)
preds_test = modelo.predict(X_test)

In [ ]:
print("Accuracy en entrenamiento:", accuracy_score(y_train, preds_train))
print("Accuracy en prueba:", accuracy_score(y_test, preds_test))

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, preds_test).ravel()
print("Matriz de confusión:")
print(confusion_matrix(y_test, preds_test))

In [ ]:
print("Reporte de clasificación:")
print(classification_report(y_test, preds_test))

In [ ]:
sns.heatmap(confusion_matrix(y_test, preds_test), annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusión')
plt.xlabel('Predicción')
plt.ylabel('Realidad')
plt.show()

In [ ]:
corr = df.corr()
plt.figure(figsize=(10, 6))
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Matriz de Correlación')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df['pasajeros'], kde=True)
plt.title('Distribución de Pasajeros')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='asientos', y='pasajeros', data=df)
plt.title('Relación entre Asientos y Pasajeros')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x=df['pasajeros'])
plt.title('Boxplot de Pasajeros')
plt.show()